In [7]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tsfel
import os
import pickle as pkl

# Constants
time = 10
offset = 100
folders = ["LAYING","SITTING","STANDING","WALKING","WALKING_DOWNSTAIRS","WALKING_UPSTAIRS"]
classes = {"WALKING":1,"WALKING_UPSTAIRS":2,"WALKING_DOWNSTAIRS":3,"SITTING":4,"STANDING":5,"LAYING":6}

dir = os.path.join("./HAR/Wild Data")

X = []
y = []

for folder in folders:
    files = os.listdir(os.path.join(dir,folder))

    for file in files:

        df = pd.read_csv(os.path.join(dir,folder,file),sep=",",header=0)
        df = df[offset:offset+time*50]
        X.append(df.values)
        y.append(classes[folder])

print("Wild data shape: ",len(X))

Wild data shape:  18


In [ ]:
clf1 = pkl.load("raw.pkl")

y_pred1 = clf1.predict(X)


accuracy = accuracy_score(y, y_pred1)
precision = precision_score(y, y_pred1, average='weighted')
recall = recall_score(y, y_pred1, average='weighted')
cm = confusion_matrix(y, y_pred1)

In [ ]:
clf2 = pkl.load("tsfel.pkl")

y_pred1 = clf1.predict(X)


accuracy = accuracy_score(y, y_pred1)
precision = precision_score(y, y_pred1, average='weighted')
recall = recall_score(y, y_pred1, average='weighted')
cm = confusion_matrix(y, y_pred1)

In [ ]:
clf3 = pkl.load("features.pkl")

y_pred1 = clf1.predict(X)


accuracy = accuracy_score(y, y_pred1)
precision = precision_score(y, y_pred1, average='weighted')
recall = recall_score(y, y_pred1, average='weighted')
cm = confusion_matrix(y, y_pred1)